# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [56]:
import pandas as pd
import numpy as np
import requests
import math
from scipy import stats
import pprint

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [57]:
stocks = pd.read_csv('data/sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [58]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
pprint.pprint(data)

{'avg10Volume': 105385817,
 'avg30Volume': 99336536,
 'beta': 1.1524296552874858,
 'companyName': 'Apple Inc',
 'day200MovingAvg': 109.6,
 'day30ChangePercent': 0.04460905141831622,
 'day50MovingAvg': 122.26,
 'day5ChangePercent': 0.0405718618344107,
 'dividendYield': 0.006759269876505964,
 'employees': 0,
 'exDividendDate': '2020-10-26',
 'float': 0,
 'marketcap': 2109974268552,
 'maxChangePercent': 47.20098254580603,
 'month1ChangePercent': 0.04429383937072964,
 'month3ChangePercent': 0.02531912590567381,
 'month6ChangePercent': 0.512097366647569,
 'nextDividendDate': '0',
 'nextEarningsDate': '0',
 'peRatio': 37.4220408060245,
 'sharesOutstanding': 17393206937,
 'ttmDividendRate': 0.8350704523586723,
 'ttmEPS': 3.32,
 'week52change': 0.90855345928282,
 'week52high': 134.79,
 'week52low': 56.37,
 'year1ChangePercent': 0.846529946174356,
 'year2ChangePercent': 2.0103870440937195,
 'year5ChangePercent': 3.660675349170189,
 'ytdChangePercent': 0.7207636883556398}


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [59]:
data['year1ChangePercent']

0.846529946174356

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [60]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [61]:
final_dataframe = pd.DataFrame(columns=my_columns)

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [62]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, data[symbol]['quote']['latestPrice'],data[symbol]['stats']['year1ChangePercent'], 'N/A'],index = my_columns),
                                        ignore_index = True) 
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,116.10,0.423597,N/A
1,AAL,17.44,-0.379485,N/A
2,AAP,153.90,0.000511547,N/A
3,AAPL,129.25,0.883194,N/A
4,ABBV,106.99,0.288041,N/A
...,...,...,...,...
500,YUM,108.51,0.0872698,N/A
501,ZBH,149.35,-0.0122246,N/A
502,ZBRA,390.63,0.463304,N/A
503,ZION,43.20,-0.105962,N/A


In [63]:
final_dataframe.sort_values(['One-Year Price Return'], ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [64]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Please enter the portfolio size: ")
    try:
        portfolio_size = float(portfolio_size)
    except ValueError:
        print("Please enter a floating point number ")

In [65]:
portfolio_input()

Please enter the portfolio size: 10000000


In [66]:
position_size = math.floor(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = portfolio_size/final_dataframe.loc[i, "Price"]
final_dataframe

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,37.46,2.14482,266951
1,345,NVDA,544.68,1.57799,18359.4
2,31,AMD,95.31,1.42227,104921
3,275,LB,40.28,1.31728,248262
4,23,ALB,140.00,1.23658,71428.6
5,179,FCX,25.53,1.16106,391696
6,387,PYPL,221.65,1.1311,45116.2
7,339,NOW,541.45,0.993549,18468.9
8,388,QCOM,161.25,0.977617,62015.5
9,180,FDX,307.74,0.947628,32495


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [70]:
hqm_columns = ['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return', 'One-Year Return Percentile', 
               'Six-Month Price Return', 'Six-Month Return Percentile', 'Three-Month Price Return', 
               'Three-Month Return Percentile', 'One-Month Price Return', 'One-Month Return Percentile','HQM Score']

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series([symbol, 
                   data[symbol]['quote']['latestPrice'],
                   'N/A',
                   data[symbol]['stats']['year1ChangePercent'],
                   'N/A',
                   data[symbol]['stats']['month6ChangePercent'],
                   'N/A',
                   data[symbol]['stats']['month3ChangePercent'],
                   'N/A',
                   data[symbol]['stats']['month1ChangePercent'],
                   'N/A',
                   'N/A'
],
                 index=hqm_columns),
        ignore_index=True)

In [72]:
hqm_dataframe.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.10,N/A,0.423121,N/A,0.280488,N/A,0.184137,N/A,0.0167334,N/A,N/A
1,AAL,17.39,N/A,-0.388361,N/A,-0.0752295,N/A,0.277701,N/A,0.516964,N/A,N/A
2,AAP,155.55,N/A,0.000518114,N/A,0.0561567,N/A,0.00120534,N/A,-0.0178595,N/A,N/A
3,AAPL,127.68,N/A,0.882174,N/A,0.521848,N/A,0.0259617,N/A,0.0429723,N/A,N/A
4,ABBV,110.28,N/A,0.285899,N/A,0.163157,N/A,0.169015,N/A,0.14471,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [76]:
time_periods = ['One-Year','Six-Month','Three-Month','One-Month']

for i in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[i, f"{time_period} Return Percentile"] = stats.percentileofscore(hqm_dataframe[f"{time_period} Return Percentile"],
                                                                                          hqm_dataframe.loc[i, f"{time_period} Return Percentile"])

In [77]:
hqm_dataframe.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.10,N/A,0.423121,50.099,0.280488,50.099,0.184137,50.099,0.0167334,50.099,N/A
1,AAL,17.39,N/A,-0.388361,50,-0.0752295,50,0.277701,50,0.516964,50,N/A
2,AAP,155.55,N/A,0.000518114,49.901,0.0561567,49.901,0.00120534,49.901,-0.0178595,49.901,N/A
3,AAPL,127.68,N/A,0.882174,49.802,0.521848,49.802,0.0259617,49.802,0.0429723,49.802,N/A
4,ABBV,110.28,N/A,0.285899,49.703,0.163157,49.703,0.169015,49.703,0.14471,49.703,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [79]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [80]:
hqm_dataframe.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.10,N/A,0.423121,50.099,0.280488,50.099,0.184137,50.099,0.0167334,50.099,50.099
1,AAL,17.39,N/A,-0.388361,50,-0.0752295,50,0.277701,50,0.516964,50,50
2,AAP,155.55,N/A,0.000518114,49.901,0.0561567,49.901,0.00120534,49.901,-0.0178595,49.901,49.901
3,AAPL,127.68,N/A,0.882174,49.802,0.521848,49.802,0.0259617,49.802,0.0429723,49.802,49.802
4,ABBV,110.28,N/A,0.285899,49.703,0.163157,49.703,0.169015,49.703,0.14471,49.703,49.703


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [87]:
hqm_dataframe.sort_values("HQM Score", ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index()

<ipython-input-87-bc6286529da5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values("HQM Score", ascending=False, inplace=True)


,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,0,A,118.100,N/A,0.423121,50.099,0.280488,50.099,0.184137,50.099,0.0167334,50.099,50.099
1,1,AAL,17.390,N/A,-0.388361,50,-0.0752295,50,0.277701,50,0.516964,50,50
2,2,AAP,155.550,N/A,0.000518114,49.901,0.0561567,49.901,0.00120534,49.901,-0.0178595,49.901,49.901
3,3,AAPL,127.680,N/A,0.882174,49.802,0.521848,49.802,0.0259617,49.802,0.0429723,49.802,49.802
4,4,ABBV,110.280,N/A,0.285899,49.703,0.163157,49.703,0.169015,49.703,0.14471,49.703,49.703
5,5,ABC,99.510,N/A,0.153428,49.604,0.01384,49.604,0.0346728,49.604,-0.039389,49.604,49.604
6,6,ABMD,283.130,N/A,0.450952,49.505,0.0901446,49.505,-0.0152274,49.505,-0.00122105,49.505,49.505
7,7,ABT,107.300,N/A,0.281241,49.4059,0.209859,49.4059,0.0333544,49.4059,-0.0650284,49.4059,49.4059
8,8,ACN,259.400,N/A,0.257632,49.3069,0.213885,49.3069,0.0603414,49.3069,0.0551228,49.3069,49.3069
9,9,ADBE,502.550,N/A,0.62942,49.2079,0.261752,49.2079,0.000644969,49.2079,-0.00502018,49.2079,49.2079


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [88]:
portfolio_input()

Please enter the portfolio size: 10000000


In [89]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.100,1693,0.423121,50.099,0.280488,50.099,0.184137,50.099,0.0167334,50.099,50.099
1,AAL,17.390,11500,-0.388361,50,-0.0752295,50,0.277701,50,0.516964,50,50
2,AAP,155.550,1285,0.000518114,49.901,0.0561567,49.901,0.00120534,49.901,-0.0178595,49.901,49.901
3,AAPL,127.680,1566,0.882174,49.802,0.521848,49.802,0.0259617,49.802,0.0429723,49.802,49.802
4,ABBV,110.280,1813,0.285899,49.703,0.163157,49.703,0.169015,49.703,0.14471,49.703,49.703
5,ABC,99.510,2009,0.153428,49.604,0.01384,49.604,0.0346728,49.604,-0.039389,49.604,49.604
6,ABMD,283.130,706,0.450952,49.505,0.0901446,49.505,-0.0152274,49.505,-0.00122105,49.505,49.505
7,ABT,107.300,1863,0.281241,49.4059,0.209859,49.4059,0.0333544,49.4059,-0.0650284,49.4059,49.4059
8,ACN,259.400,771,0.257632,49.3069,0.213885,49.3069,0.0603414,49.3069,0.0551228,49.3069,49.3069
9,ADBE,502.550,397,0.62942,49.2079,0.261752,49.2079,0.000644969,49.2079,-0.00502018,49.2079,49.2079


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [90]:
hqm_dataframe.to_csv("data/hqm_picks.csv", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: